# Load and Combine Data

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import pointbiserialr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Load four data sources
df = pd.read_csv("HR Analytics Case Study/general_data.csv")
employee_survey = pd.read_csv("HR Analytics Case Study/employee_survey_data.csv")
manager_survey = pd.read_csv("HR Analytics Case Study/manager_survey_data.csv")
worktime = pd.read_csv("HR Analytics Case Study/work_time.csv")

# Set primary key and map Attrition values to binary
primary_col = "EmployeeID"
target_col = "Attrition"
df[target_col] = df[target_col].map({'Yes': 1, 'No': 0})

# Set index for merging
df.set_index(primary_col, inplace=True)
worktime.set_index(primary_col, inplace=True)
employee_survey.set_index(primary_col, inplace=True)
manager_survey.set_index(primary_col, inplace=True)

# Combine all dataframes
merged_df = df.join([worktime, employee_survey, manager_survey], how="outer").reset_index()

In [2]:
print(merged_df.head())

   EmployeeID  Age  Attrition     BusinessTravel              Department  \
0           1   51          0      Travel_Rarely                   Sales   
1           2   31          1  Travel_Frequently  Research & Development   
2           3   32          0  Travel_Frequently  Research & Development   
3           4   38          0         Non-Travel  Research & Development   
4           5   32          0      Travel_Rarely  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  Gender  ...  \
0                 6          2  Life Sciences              1  Female  ...   
1                10          1  Life Sciences              1  Female  ...   
2                17          4          Other              1    Male  ...   
3                 2          5  Life Sciences              1    Male  ...   
4                10          1        Medical              1    Male  ...   

   TrainingTimesLastYear YearsAtCompany YearsSinceLastPromotion  \
0            

# Handle Missing Values

In [3]:
# Check for missing values
missing_values = merged_df.isnull().sum()
missing_values[missing_values > 0].sort_values(ascending=False)

WorkLifeBalance            38
EnvironmentSatisfaction    25
JobSatisfaction            20
NumCompaniesWorked         19
TotalWorkingYears           9
dtype: int64

In [4]:
# print type of columns
print(merged_df.dtypes)

EmployeeID                   int64
Age                          int64
Attrition                    int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike            int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
Avg_Work_Hours             float64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
JobInvolvement      

In [5]:
# Fill missing values by type
numerical_cols =merged_df.select_dtypes(include=['number']).columns.tolist()
categorical_cols = merged_df.select_dtypes(include=['object', 'category']).columns.tolist()
for column in categorical_cols:
    merged_df[column] = merged_df[column].fillna(merged_df[column].mode()[0])
for column in numerical_cols:
    merged_df[column] = merged_df[column].fillna(merged_df[column].mean())

# Statistical Correlation Analysis (Chi2 & Point-Biserial)

In [6]:
# Define a function to assess correlation significance for numeric & categorical columns
def correlation_significance_analysis(df, key_column):
    results = []

    
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Remove key column from analysis
    numeric_cols = [col for col in numeric_cols if col != key_column]
    categorical_cols = [col for col in categorical_cols if col != key_column]

    # Pointbiserial
    from scipy.stats import pointbiserialr
    for col in numeric_cols:
        corr, p_value = pointbiserialr(df[col], df[key_column])
        results.append({
            'Feature': col,
            'Type': 'Numeric',
            'Score': corr,
            'P-value': p_value,
            'Is Significant': p_value < 0.05
        })

    # chi2_contingency
    from scipy.stats import chi2_contingency
    for col in categorical_cols:
        contingency = pd.crosstab(df[col], df[key_column])
        chi2, p, dof, expected = chi2_contingency(contingency)
        results.append({
            'Feature': col,
            'Type': 'Categorical',
            'Score': chi2,
            'P-value': p,
            'Is Significant': p < 0.05
        })

    return pd.DataFrame(results).sort_values("P-value")


In [7]:
# Perform correlation significance analysis
correlation_significance_results = correlation_significance_analysis(merged_df,key_column='Attrition')
print(correlation_significance_results)

c:\Users\NCPC\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


                    Feature         Type       Score       P-value  \
16           Avg_Work_Hours      Numeric    0.201683  1.057946e-41   
27            MaritalStatus  Categorical  138.491030  8.453859e-31   
11        TotalWorkingYears      Numeric   -0.170111  5.473148e-30   
1                       Age      Numeric   -0.159205  1.996802e-26   
15     YearsWithCurrManager      Numeric   -0.156199  1.733932e-25   
13           YearsAtCompany      Numeric   -0.134392  3.163883e-19   
22           BusinessTravel  Categorical   72.547241  1.764277e-16   
18          JobSatisfaction      Numeric   -0.102743  7.967829e-12   
17  EnvironmentSatisfaction      Numeric   -0.101795  1.238112e-11   
24           EducationField  Categorical   46.194921  8.288917e-09   
23               Department  Categorical   29.090275  4.820888e-07   
19          WorkLifeBalance      Numeric   -0.062561  3.218590e-05   
12    TrainingTimesLastYear      Numeric   -0.049431  1.024706e-03   
26                  

# Select Statistically Significant Features and Encode Categorical Variables

In [8]:
# Select Statistically Significant Features
significant_features = correlation_significance_results[correlation_significance_results['Is Significant'] == True]

# Columns classification
numerical_cols = significant_features[significant_features['Type'] == 'Numeric']['Feature'].tolist()
categorical_cols = significant_features[significant_features['Type'] == 'Categorical']['Feature'].tolist()

# Encode categorical feature using Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col])
    label_encoders[col] = le
merged_df = merged_df[[primary_col, target_col] + numerical_cols + categorical_cols]

In [9]:
merged_df.dtypes

EmployeeID                   int64
Attrition                    int64
Avg_Work_Hours             float64
TotalWorkingYears          float64
Age                          int64
YearsWithCurrManager         int64
YearsAtCompany               int64
JobSatisfaction            float64
EnvironmentSatisfaction    float64
WorkLifeBalance            float64
TrainingTimesLastYear        int64
NumCompaniesWorked         float64
YearsSinceLastPromotion      int64
PercentSalaryHike            int64
MonthlyIncome                int64
MaritalStatus                int32
BusinessTravel               int32
EducationField               int32
Department                   int32
JobRole                      int32
dtype: object

In [10]:
merged_df.columns

Index(['EmployeeID', 'Attrition', 'Avg_Work_Hours', 'TotalWorkingYears', 'Age',
       'YearsWithCurrManager', 'YearsAtCompany', 'JobSatisfaction',
       'EnvironmentSatisfaction', 'WorkLifeBalance', 'TrainingTimesLastYear',
       'NumCompaniesWorked', 'YearsSinceLastPromotion', 'PercentSalaryHike',
       'MonthlyIncome', 'MaritalStatus', 'BusinessTravel', 'EducationField',
       'Department', 'JobRole'],
      dtype='object')

In [11]:
# Check for missing values
missing_values = merged_df.isnull().sum()
missing_values[missing_values > 0].sort_values(ascending=False)


Series([], dtype: int64)

# Split Data: 80% train, 20% test

In [12]:
# Define features (X) and target (y)
X = merged_df.drop(columns=[target_col, primary_col])  # Feature columns
X = pd.get_dummies(X, drop_first=False)
y = merged_df[target_col]  # Target column

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train RandomForestClassifier Model

In [13]:
# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Predict labels on the test set and Print classification report

In [14]:
# Predict labels on the test set
y_pred = model.predict(X_test)
cf = confusion_matrix(y_test, y_pred)

# Print classification report (precision, recall, f1-score, support)
result = classification_report(y_test, y_pred, target_names=["Attrition: No", "Attrition: Yes"])
print(result)

                precision    recall  f1-score   support

 Attrition: No       0.99      1.00      1.00       741
Attrition: Yes       1.00      0.96      0.98       141

      accuracy                           0.99       882
     macro avg       1.00      0.98      0.99       882
  weighted avg       0.99      0.99      0.99       882



# Predict Attrition Risk for Active Employees

In [15]:
# Filter employees who have not left the company
no_attrition = merged_df[merged_df[target_col] == 0].copy()

# Drop target and primary ID column for prediction
current_data = no_attrition.drop(columns=[target_col, primary_col])

# Predict probability of attrition (class = 1) for each active employee
no_attrition['Attrition_Probability'] = model.predict_proba(current_data)[:, 1]

# Display top 5 employees with the highest attrition risk
print(no_attrition.nlargest(5, 'Attrition_Probability')[['EmployeeID', 'Attrition_Probability']])

      EmployeeID  Attrition_Probability
415          416                   0.35
1885        1886                   0.35
3355        3356                   0.35
426          427                   0.34
877          878                   0.33


# Export Prediction Results to CSV

In [16]:
# Keep only essential columns: EmployeeID and predicted attrition probability
export_df = no_attrition[[primary_col, 'Attrition_Probability']].copy()

# Export to CSV for Power BI dashboard visualization
export_df.to_csv('predicted_attrition_risk.csv', index=False)
print("Exported to 'predicted_attrition_risk.csv' successfully!")

Exported to 'predicted_attrition_risk.csv' successfully!
